# Environment

In [10]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import json
import urllib
import re
import os

# Function

In [11]:
def RequestFilmInfo(film_id):
  api_key = "0b2bdeda43b5688921839c8ecb20399b"
  city = "shanghai"
  client = "Nothing"
  udid = "Nothing"
  request_url_fmt = "http://api.douban.com/v2/movie/subject/%s?apikey=%s&city=%s&client=%s&udid=%s"
  r = urllib.request.urlopen(request_url_fmt % (film_id, api_key, city, client, udid))
  return r.read()

In [12]:
def GetFilmIdFromUrl(url):
  try:
    result = re.fullmatch("https://movie.douban.com/subject/(.\\d+)/", url)
    return result[1]
  except:
    return False

In [18]:
def ScanDouban():
  # Film list
  with open("data/films.json", "rb") as f:
    film_list_raw = f.read()
  film_list = []
  for item in film_list_raw.split(b"\n"):
    try:
      json.loads(item)
      film_list.append(json.loads(item))
    except:
      pass
  
  # Get Film Info
  import tqdm
#   import time
#   import random

  np.random.shuffle(film_list)
  
  for film in tqdm.tqdm_notebook(film_list):
    film_id = GetFilmIdFromUrl(film["film_page"])
    json_path = "data/each_film/%s.json" % (film_id)
    if film_id is False or os.path.exists(json_path):
      continue
    else:
      try:
        film_info = RequestFilmInfo(film_id)
#         time.sleep(random.randint(0, 5))
        with open(json_path, "wb") as f:
          f.write(film_info)      
      except:
        with open(json_path, "wb") as f:
          f.write(b"error")

# Dev

In [17]:
import glob
import tqdm
for file in tqdm.tqdm_notebook(glob.glob("data/each_film/*.json")):
  to_remove = False
  with open(file, "rb") as f:
    if f.read() == b"error":
      to_remove = True
  if to_remove:
    os.remove(file)

# Run

In [19]:
ScanDouban()

# View